In [2]:
%autoreload 2

from VMTest import *

In [4]:
database_name = 'allEdges'
completed = checkCompleted(database_name)
completed

['a']

In [5]:
runStats('Frat')

that took 0.714102029800415


In [15]:
letters = list(map(chr, range(97, 123)))
numbers = list(range(0,10))
chars = letters + numbers

database_name = 'allEdges'
completed = checkCompleted(database_name)

for char in chars:
    if char in completed:
        print(char, 'already done')
    else:
        print(char)
        query = """
                    SELECT author, subreddit, count(link_id) as weight
                    FROM `fh-bigquery.reddit_comments.2018_03`
                    WHERE lower(author) like '{}%' 
                    GROUP BY author, subreddit
                """.format(char)

        data = fetchQuery(query)
        database_name = 'allEdges'
        saveSQL(data, char, database_name)

In [ ]:
query = """
            SELECT author, subreddit, count(link_id) as weight
            FROM `fh-bigquery.reddit_comments.2018_03`
            WHERE (author not like '[A-Z]%') AND
                  (author not like '[0-9]%')
            GROUP BY author, subreddit
        """.format(startswith)

data = fetchQuery(query)

saveSQL(data, 'punc', database_name)

In [1]:
completed = checkCompleted(database_name)
completed

NameError: name 'checkCompleted' is not defined

In [ ]:
punc = loadSQL('punc', database_name, index_col='index')
punc.head()